In [16]:
import numpy as np
import matplotlib as plt

class coin_model:
    def __init__(self, p, q):
        self.coins_H_p = [p, q]
    
    def throw_coin(self, coin_num):
        p = self.coins_H_p[coin_num]
        val = np.random.random()
        if val <= p:
            return 1
        return 0
    
    """ Generate the sample data for the current distribution.
    """
    def generate_sample(self, times, default_per_times=10):
        sample = np.zeros((times, default_per_times+1))
        for i in range(times):
            a = 0 if np.random.random() <= 0.5 else 1
            for j in range(default_per_times+1):
                sample[i][j] = self.throw_coin(a)
            sample[i][default_per_times] = a
        return sample

The following is just a test for the result from above.  
The last column indicate which coin we are throwing.

In [17]:
c = coin_model(0.3, 0.6)
sample = c.generate_sample(6)
print(sample)

[[0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 1.]
 [1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 1.]
 [1. 1. 0. 1. 1. 1. 0. 0. 1. 0. 1.]
 [1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0.]
 [1. 0. 1. 0. 0. 0. 1. 1. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0.]]


Now, we want to apply the EM Algorithm to this model. We want to guess the p(H|coin=0) and p(H|coin=1). But imagine if we don't konw which coin we are throwing each row, then we have to guess.

According to the EM Algorithm, we want to maximize $$\mathcal{l}(x|\theta) = \log P(x|\theta)$$  
By definition, we can see that
\begin{align*}
\log P(x|\theta) 
&= \log \prod_{i=1}^m P(x_i|\theta)\\
&= \sum_{i=1}^m \log P(x_i|\theta)\\
&= \sum_{i=1}^m \log \sum_c P(x_i, c|\theta)\\
&= \sum_{i=1}^m \log \sum_c Q_i(c)\cdot \frac{P(x_i, c|\theta)}{Q_i(c)}\\
&\geq \sum_{i=1}^m \sum_c Q_i(c) \log \frac{P(x_i, c|\theta)}{Q_i(c)} \quad\text{By $\log(E(x))\geq E(\log(x))$}
\end{align*}
Now, consider $\frac{P(x_i, c|\theta)}{Q_i(c)}$, if we can make it be a function that isn't influenced by c, we would have the equality. So let $Q_i(c)=P(c|x_i,\theta)$, we can see that $$\frac{P(x_i, c|\theta)}{Q_i(c)} = P(x_i|\theta)$$
We can use the Bayes' Rule to calculate $Q_i(c)$
\begin{align*}
Q_i(c)
&= P(c|x_i,\theta)\\
&= \frac{P(c, x_i|\theta)}{P(x_i)}\\
&= \frac{P(c, x_i|\theta)}{\sum_c P(x_i, c|\theta)}\\
&= \frac{P(x_i|c, \theta)P(c|\theta)}{\sum_c P(x_i|c, \theta)P(c|\theta)}
\end{align*}